# Anomaly Detection (Binary 2D)

Discuss dataset here including shape. Also discuss which data set you specifically chose here.


**Inputs**  


The 14 waveforms (features) reported in the dataset in order are:

- `A+IGBT-I`: Current passing through the IGBT switch of phase A+ in Qa1 ($A$).  
- `A+*IGBT-I`: Current passing through the IGBT switch of phase A+\* in Qa3 ($A$).  
- `B+IGBT-I`: Current passing through the IGBT switch of phase B+ in Qb1 ($A$).  
- `B+*IGBT-I`: Current passing through the IGBT switch of phase B+\* in Qb3 ($A$).  
- `C+IGBT-I`: Current passing through the IGBT switch of phase C+ in Qc1 ($A$).  
- `C+*IGBT-I`: Current passing through the IGBT switch of phase C+\* in Qc3 ($A$).  
- `A-Flux`: Magnetic flux density for phase A in transformer XA ($-$).  
- `B-Flux`: Magnetic flux density for phase B in transformer XB ($-$).  
- `C-Flux`: Magnetic flux density for phase C in transformer XC ($-$).  
- `Mod-V`: Modulator voltage ($V$).  
- `Mod-I`: Modulator current ($A$).  
- `CB-I`: Cap bank current ($-$).  
- `CB-V`: Cap bank voltage ($V$).  
- `DV/DT`: Time derivative change of the Mod\-V voltage ($-$).

**Outputs**
- `Fault`: Faulty wave pulse
- `Normal` Normal wave pulse

**Shaping**
Outputs are 1x3 arrays, with the first element being occurrences, second being timesteps, and third being `Fault` or `Run`

![Image](display.jpg)




**Files and Descriptions**

The files provided in the dataset are:

- `RFQ.npy`: 3D numpy array of the normal and faulty pulses for the RFQ system (shape: (872, 4500, 14)).
- `RFQ_labels.npy`: 2D numpy array of the labels for the RFQ system (shape: (872, 3)).
- `DTL.npy`: 3D numpy array of the normal and faulty pulses for the DTL system (shape: (1077, 4500, 14)).
- `DTL_labels.npy`: 2D numpy array of the labels for the DTL system (shape: (1077, 3)).
- `CCL.npy`: 3D numpy array of the normal and faulty pulses for the CCL system (shape: (2057, 4500, 14)).
- `CCL_labels.npy`: 2D numpy array of the labels for the CCL system (shape: (2057, 3)).
- `SCL.npy`: 3D numpy array of the normal and faulty pulses for the SCL system (shape: (4598, 4500, 14)).
- `SCL_labels.npy`: 2D numpy array of the labels for the SCL system (shape: (4598, 3)).

High Voltage Converter Modulators (HVCM) have frequent failures, making them a major source of down time for the Spallation neutron source. HVCMs are well instrumented to collect large amounts of waveform data including but not limited to modulator current, modulator voltage, magnetic flux, cap bank voltage, cap bank current, and others. These waveform data are collected from all 15 HVCM systems for the application of anomaly detection and failure prediction.

All 15 modulators are grouped into four major subsystems based on their type as follows: 1 modulator as radio-frequency quadrupole (RFQ), 2 modulators as drift-tube linac (DTL), 4 modulators as coupled-cavity linac (CCL), and 8 modulators as super-conducting linac (SCL). A waveform is featured by consecutive pulses, which could be either a normal pulse if the system is healthy, or anomaly pulse if the system is close to failing. These pulses are collected and reported in the dataset of this paper, where the details of the data collection and processing are described in the next section.

The researchers will select the DTL dataset

![Image](data.jpg) fxample.

4598, 3)

In [1]:
import pickle
import sys
import matplotlib.pyplot as plt
sys.path.insert(0, "supporting/anomaly")

from supporting.anomaly.preprocessing import load_anomaly_data
import supporting.anomaly.settings as anomaly_settings
import pyMAISE as mai

2024-06-23 21:09:54.759080: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 21:09:54.759118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 21:09:54.760562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 21:09:54.767471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 21:09:55.749510: W tensorflow/compiler/tf2

## pyMAISE Initialization
In the settings file, the researchers initialize pyMAISE as a classification problem and load the DTL data set. GPU is defined and utilized for LSTM and GRU models.

In [2]:
global_settings = mai.init(
    problem_type=anomaly_settings.problem_type,
    random_state=anomaly_settings.random_state,
    cuda_visible_devices="1",  # Use GPU 1
)

The settings file defines a 70/30 training and testing split. To train the machine learning models the researchers desire roughly a 50/50 split between `Fault` and `Normal`. There were significantly more `Normal` cases, so these `normal` datapoints were removed at random points to preserve the sequential pattern of the time dataset.

![Frequency of positive/negative values in anomaly detection data.](supporting/anomaly/figs/bc2_frequency.png)

This plot shows the frequency of `Normal` and `Fault` cases with the trimmed positive data.


In [3]:
xtrain, xtest, ytrain, ytest, xscaler = load_anomaly_data(
    global_settings=global_settings,
    stack_series=True,
    multiclass=False,
    test_size=anomaly_settings.test_size,
    non_faulty_frac=anomaly_settings.non_faulty_frac,
    timestep_step=1,
)

xtrain shape: (303, 4500, 14)
xtest shape: (131, 4500, 14)
ytrain shape: (303, 4500, 2)
ytest shape: (131, 4500, 2)


In [4]:
xtrain

<xarray.DataArray (samples: 303, time steps: 4500, features: 14)>
array([[[0.6002122 , 0.60862154, 0.80283475, ..., 0.50022596,
         0.9528328 , 0.4937436 ],
        [0.60082346, 0.60862154, 0.8020122 , ..., 0.49930644,
         0.9531946 , 0.48889887],
        [0.6005178 , 0.6083029 , 0.8020122 , ..., 0.49930644,
         0.9528328 , 0.4925326 ],
        ...,
        [0.6047964 , 0.61117077, 0.80489117, ..., 0.49838698,
         0.917107  , 0.52644247],
        [0.6054076 , 0.6105335 , 0.80489117, ..., 0.4974674 ,
         0.9167904 , 0.5264474 ],
        [0.60510194, 0.6108522 , 0.80489117, ..., 0.4974674 ,
         0.9167904 , 0.52281594]],

       [[0.6017403 , 0.60957754, 0.8024235 , ..., 0.50160533,
         0.9522902 , 0.49374384],
        [0.60082346, 0.6089402 , 0.8016009 , ..., 0.50022596,
         0.9516571 , 0.49616608],
        [0.6005178 , 0.6092589 , 0.8020122 , ..., 0.49930644,
         0.9526067 , 0.49616632],
...
        [0.60846364, 0.6114895 , 0.8061251 , ..., 0.49838698,
         0.8540802 , 0.51917946],
        [0.609075  , 0.61180824, 0.8061251 , ..., 0.49930644,
         0.85442394, 0.51433253],
        [0.60846364, 0.6124454 , 0.80489117, ..., 0.4974674 ,
         0.85442394, 0.52039   ]],

       [[0.60082346, 0.60862154, 0.80365735, ..., 0.50068575,
         0.95179266, 0.4985888 ],
        [0.60143465, 0.6089402 , 0.80283475, ..., 0.49930644,
         0.9520188 , 0.49616608],
        [0.60082346, 0.6089402 , 0.8016009 , ..., 0.49930644,
         0.95170224, 0.4949551 ],
        ...,
        [0.60601896, 0.6114895 , 0.8061251 , ..., 0.4956282 ,
         0.9153885 , 0.53250486],
        [0.6057133 , 0.61117077, 0.8061251 , ..., 0.4937892 ,
         0.91516244, 0.5300789 ],
        [0.60510194, 0.61117077, 0.8069476 , ..., 0.4956282 ,
         0.91561466, 0.53250486]]], dtype=float32)
Coordinates:
  * samples     (samples) int64 118 114 140 137 322 332 ... 71 106 270 348 102
  * time steps  (time steps) int64 0 1 2 3 4 5 ... 4494 4495 4496 4497 4498 4499
  * features    (features) <U18 'A+IGBT-I: current' ... 'DV/DT'

In [5]:
ytrain

<xarray.DataArray (samples: 303, time steps: 4500, features: 2)>
array([[[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]],

       [[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]],

       [[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
...
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]]])
Coordinates:
  * samples     (samples) int64 118 114 140 137 322 332 ... 71 106 270 348 102
  * time steps  (time steps) int64 0 1 2 3 4 5 ... 4494 4495 4496 4497 4498 4499
  * features    (features) object 'Class_Fault' 'Class_Run'

## Model Initialization and Hyperparameter Tuning

The researchers will compare the performance of 2 machine learning (ML) models:

- Long short-term memory: `LSTM`,
- Gated recurrent unit: `GRU`,

Both `LSTM` and `GRU` utilize input, output, and dense input/output layers, in order. The researchers define the hyperparameter search objective as accuracy, utilizing Bayesian optimization and 50 trials. This is used to find the best hyperparameters that maximize accuracy. The results are saved using `pickle` and imported to Jupyter.

![Convergence of Bayesian optimizer to best hyperparameter configuration.](supporting/anomaly/figs/bc2_convergence.png)

The search could not produce an LSTM/GRU configuration with performance better than ~0.60.

In [6]:
with open("supporting/anomaly/configs/binary_case_1.pkl", "rb") as f:
    configs = pickle.load(f)

2024-06-23 21:09:57.722497: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-23 21:09:57.757142: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-23 21:09:57.757424: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## Model Postprocessing

With the models tuned and the top hyperparameters saved, we can now pass these models to the `pyMAISE.PostProcessor` for model comparison and analysis. We will increase the `LSTM` and `GRU` models epochs for better performance.

In [7]:
postprocessor = mai.PostProcessor(
    data=(xtrain, xtest, ytrain, ytest),
    model_configs=[configs],
    new_model_settings={
        "LSTM": {"fitting_params": {"epochs": 100}},
        "GRU": {"fitting_params": {"epochs": 100}},
        "CNN-LSTM": {"fitting_params": {"epochs": 100}},
    },
)

ValueError: in user code:

    File "/home/jacc/anaconda3/envs/tf2.15/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/jacc/anaconda3/envs/tf2.15/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jacc/anaconda3/envs/tf2.15/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/jacc/anaconda3/envs/tf2.15/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/jacc/anaconda3/envs/tf2.15/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/home/jacc/anaconda3/envs/tf2.15/lib/python3.11/si

To compare the performance of these models, we will compute four metrics for both the training and testing data: 
- `Accuracy` = $\frac{\text{Number of Correct Predictions}}{\text{Number of Instances}}$
- `Recall`= $\frac{\text{True Positives}}{\text{True Positives}+\text{False Negatives}}$
- `Precision` = $\frac{\text{True Positives}}{\text{True Positives}+\text{False Positives}}$
- `F1`= $\frac{2*\text{Precision}*\text{Recall}}{\text{Precision}+\text{Recall}}$

In [ ]:
postprocessor.metrics()

Based on the performance above, the LSTM model is the top performing model. The top 3 models are LSTM, and their corresponding accuracy scores are in the low 60\% and high 50\% respectively. GRU struggled to produce a higher accuracy than 57%, with an average being around 54%. 

Next, the best-performing model hyperparameters are shown.

In [ ]:
for model in ["LSTM", "GRU", "CNN-LSTM"]:
    for key, value in postprocessor.get_params(model_type=model).to_dict().items():
        print(f"{key}: {value[0]}")
    print()

In [ ]:
postprocessor.confusion_matrix(model_type="LSTM", figsize=(15, 5))
plt.show()

In [ ]:
postprocessor.confusion_matrix(model_type="GRU", figsize=(15, 5))
plt.show()

In [ ]:
postprocessor.confusion_matrix(model_type="CNN-LSTM", figsize=(15, 5))
plt.show()

Discuss confusion matrix results.

In [ ]:
postprocessor.nn_learning_plot(model_type="LSTM")
plt.show()

In [ ]:
postprocessor.nn_learning_plot(model_type="GRU")
plt.show()

In [ ]:
postprocessor.nn_learning_plot(model_type="CNN-LSTM")
plt.show()

Discuss neural network learning curves.

![Image](supporting/anomaly/figs/pyMAISElogo.png)